In [1]:
# path='d:/My Python/'
path = 'C:/Users/HSIU/Desktop/108-1/python_ntu'

In [2]:
# 如果正向文章與負向文章，以文字檔形式，分別放在不同的資料夾，
def text():
     f1 = open(path+'pos.txt','r',encoding='utf-8') 
     f2 = open(path+'neg.txt','r',encoding='utf-8')
     line1 = f1.readline()
     line2 = f2.readline()
     str = ''
     while line1:
         str += line1
         line1 = f1.readline()
     while line2:
         str += line2
         line2 = f2.readline()
     f1.close()
     f2.close()
     return str
try:
    text=text()
except:
    pass

In [3]:
# 正向負向文本併在一起，尚未斷詞
print (text)    

<function text at 0x000001558D9D5AE8>


In [4]:
# pip install nltk
import nltk
from nltk.collocations import  BigramCollocationFinder
from nltk.metrics import  BigramAssocMeasures

In [5]:
# 結巴斷詞
import jieba
def read_file(filename):
     stop = [line.strip() for line in  open(path+'stop.txt','r',encoding='utf-8').readlines()]  #停用词
     f = open(filename,'r',encoding='utf-8')
     line = f.readline()
     str = []
     # 每行為一篇文章
     while line:
         s = line.split('\n')   # split based on line            
         fenci = jieba.cut(s[0],cut_all=False)  #False默认值：精准模式
         str.append(list(set(fenci)-set(stop)))  # 去除停用字，加入文字袋 list
         line = f.readline()
     return str

In [6]:
from nltk.probability import  FreqDist,ConditionalFreqDist
from nltk.metrics import  BigramAssocMeasures

In [12]:
# 每篇文章，取前面最有鑑別力的幾個字詞
def jieba_feature(number):   
     posWords = []
     negWords = []
     # 讀取檔案，每行為一篇文章，結巴斷詞，存成 list
     for items in read_file(path+'/pos.txt'):
         for item in items:
            posWords.append(item)
     for items in read_file(path+'/neg.txt'):
         for item in items:
            negWords.append(item)
            
     # 宣告函數
     word_fd = FreqDist() # 統計所有字詞的出現次數
     cond_word_fd = ConditionalFreqDist() #條件統計字頻
        
     # 正向貼文的字詞頻率
     for word in posWords:
         word_fd[word] += 1
         cond_word_fd['pos'][word] += 1
            
     # 負向貼文的字詞頻率
     for word in negWords:
         word_fd[word] += 1
         cond_word_fd['neg'][word] += 1
            
     # 正向字詞個數 
     pos_word_count = cond_word_fd['pos'].N() 
     # 負向字詞個數   
     neg_word_count = cond_word_fd['neg'].N() 
     # 所有字詞個數   
     total_word_count = pos_word_count + neg_word_count

     word_scores = {} # 計算每個字詞的鑑別力
     for word, freq in word_fd.items():
         #计算 pos, neg 的卡方统计量   
         pos_score = BigramAssocMeasures.chi_sq(cond_word_fd['pos'][word],  (freq, pos_word_count), total_word_count)         
         neg_score = BigramAssocMeasures.chi_sq(cond_word_fd['neg'][word],  (freq, neg_word_count), total_word_count) 
         #一个词的信息量等于积极卡方统计量加上消极卡方统计量
         word_scores[word] = pos_score + neg_score 
        
     #把词按信息量倒序排序。選取 number特征维度
     best_vals = sorted(word_scores.items(), key=lambda item:item[1],  reverse=True)[:number]     
     best_words = set([w for w,s in best_vals])
    
     # 傳回 dictionary，最有鑑別力的 number 個字詞 
     return dict([(word, True) for word in best_words])        


In [14]:
# 填入情感詞
def build_features():       
     # dictionary, 各篇文章的字詞(key)，是否出現 True, False (value)   
     feature = jieba_feature(300) #结巴分词，取前面 300 個最重要的字詞     
    
     # list 填入情感詞 
     posFeatures = []
     for items in read_file(path+'/pos.txt'):
         a = {}
         for item in items:
            if item in feature.keys():
                a[item]='True'
         # 正向文章填入 pos       
         posWords = [a,'pos'] 
         posFeatures.append(posWords)    
        
     negFeatures = []    
     for items in read_file(path+'/neg.txt'):
         a = {}
         for item in items:
            if item in feature.keys():
                a[item]='True'
         negWords = [a,'neg'] 
         negFeatures.append(negWords)
            
     # 傳回 list，含字詞及情感詞       
     return posFeatures,negFeatures

In [15]:
posFeatures,negFeatures =  build_features()

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/HSIU/Desktop/108-1/python_ntustop.txt'

In [12]:
posFeatures

[[{'喜歡': 'True'}, 'pos'],
 [{'。': 'True', '高興': 'True'}, 'pos'],
 [{'。': 'True', '太棒了': 'True', '真是': 'True'}, 'pos'],
 [{'。': 'True', '好': 'True', '我覺': 'True', '心情': 'True'}, 'pos'],
 [{'音樂會': 'True', '令人': 'True', '。': 'True', '昨天晚上': 'True', '感動': 'True'},
  'pos'],
 [{'最要': 'True', '。': 'True', '好': 'True', '朋友': 'True'}, 'pos']]

In [13]:
negFeatures

[[{'搭公車': 'True', '。': 'True', '討厭': 'True'}, 'neg'],
 [{'太': 'True',
   '可怕': 'True',
   '。': 'True',
   '經歷實': 'True',
   '在': 'True',
   '了': 'True'},
  'neg'],
 [{'累': 'True', '我覺': 'True', '。': 'True'}, 'neg'],
 [{'期待': 'True', '研討會': 'True', '。': 'True', '參加': 'True', '不': 'True'},
  'neg'],
 [{'喜歡': 'True', '。': 'True', '不': 'True'}, 'neg'],
 [{'競爭': 'True', '。': 'True', '手': 'True', '對': 'True'}, 'neg']]

## 比較原來之文本
* pos
* 我喜歡車子
* 早上我很高興。
* 他真是太棒了。
* 早上我覺得心情很好。
* 昨天晚上的音樂會令人感動。
* neg
* 他是我最要好的朋友。我討厭搭公車。
* 這個經歷實在太可怕了。
* 早上我覺得很累。
* 我一點也不期待參加這個研討會。
* 我不喜歡他。
* 他是我的競爭對手。

## 將含有情感詞的文章，分為訓練組與測試組

In [ ]:
from random import shuffle
#把文本的排列随机化 
shuffle(posFeatures) 
shuffle(negFeatures) 
# 設定 訓練集 [2:] 共 8 筆
train =  posFeatures[2:]+negFeatures[2:]

In [14]:
# 設定 測試集 [:2] 共4筆
test = posFeatures[:2]+negFeatures[:2]

#分离测试集合的数据和标签，便于验证和测试
data,tag = zip(*test)

## 訓練組

In [17]:
train

[[{'。': 'True', '高興': 'True'}, 'pos'],
 [{'。': 'True', '太棒了': 'True', '真是': 'True'}, 'pos'],
 [{'。': 'True', '好': 'True', '我覺': 'True', '心情': 'True'}, 'pos'],
 [{'音樂會': 'True', '令人': 'True', '。': 'True', '昨天晚上': 'True', '感動': 'True'},
  'pos'],
 [{'累': 'True', '我覺': 'True', '。': 'True'}, 'neg'],
 [{'競爭': 'True', '。': 'True', '手': 'True', '對': 'True'}, 'neg'],
 [{'喜歡': 'True', '。': 'True', '不': 'True'}, 'neg'],
 [{'太': 'True',
   '可怕': 'True',
   '。': 'True',
   '經歷實': 'True',
   '在': 'True',
   '了': 'True'},
  'neg']]

## 測試組

In [18]:
test

[[{'喜歡': 'True'}, 'pos'],
 [{'最要': 'True', '。': 'True', '好': 'True', '朋友': 'True'}, 'pos'],
 [{'搭公車': 'True', '。': 'True', '討厭': 'True'}, 'neg'],
 [{'期待': 'True', '研討會': 'True', '。': 'True', '參加': 'True', '不': 'True'},
  'neg']]

## 機械學習，計算準確度

In [23]:
train

[[{'。': 'True', '高興': 'True'}, 'pos'],
 [{'。': 'True', '太棒了': 'True', '真是': 'True'}, 'pos'],
 [{'。': 'True', '好': 'True', '我覺': 'True', '心情': 'True'}, 'pos'],
 [{'音樂會': 'True', '令人': 'True', '。': 'True', '昨天晚上': 'True', '感動': 'True'},
  'pos'],
 [{'累': 'True', '我覺': 'True', '。': 'True'}, 'neg'],
 [{'競爭': 'True', '。': 'True', '手': 'True', '對': 'True'}, 'neg'],
 [{'喜歡': 'True', '。': 'True', '不': 'True'}, 'neg'],
 [{'太': 'True',
   '可怕': 'True',
   '。': 'True',
   '經歷實': 'True',
   '在': 'True',
   '了': 'True'},
  'neg']]

In [30]:
def score(classifier,train,data):
     classifier = SklearnClassifier(classifier) 
     #训练分类器   
     classifier.train(train) 
     # data,testing and predict  
     pred = classifier.classify_many(data) #给出预测的标签
     n = 0
     s = len(pred)
     for i in range(0,s):
         if pred[i]==tag[i]:
            n = n+1
            print ('correct:', pred[i],test[i])
         else:
            print ('incorrect:',pred[i],test[i])
            pass
     return n/s, pred #分类器准确度

In [21]:
import sklearn
from nltk.classify.scikitlearn import  SklearnClassifier
from sklearn.svm import SVC, LinearSVC,  NuSVC
from sklearn.naive_bayes import  MultinomialNB, BernoulliNB
from sklearn.linear_model import  LogisticRegression
from sklearn.metrics import  accuracy_score

## 演算法1:  BernoulliNB

In [32]:
print ('BernoulliNB-------------------')
sco=score(BernoulliNB(),train,data)
print('accuracy is', sco[0])
print ('predict', sco[1])

BernoulliNB-------------------
incorrect: neg [{'喜歡': 'True'}, 'pos']
correct: pos [{'最要': 'True', '。': 'True', '好': 'True', '朋友': 'True'}, 'pos']
incorrect: pos [{'搭公車': 'True', '。': 'True', '討厭': 'True'}, 'neg']
correct: neg [{'期待': 'True', '研討會': 'True', '。': 'True', '參加': 'True', '不': 'True'}, 'neg']
accuracy is 0.5
predict ['neg', 'pos', 'pos', 'neg']


# 演算法2: MultinomiaNB

In [36]:
print ('MultinomialNB()-------------------')
sco=score(MultinomialNB(),train,data)
print('accuracy is', sco[0])
print ('predict', sco[1])

MultinomialNB()-------------------
incorrect: neg [{'喜歡': 'True'}, 'pos']
correct: pos [{'最要': 'True', '。': 'True', '好': 'True', '朋友': 'True'}, 'pos']
incorrect: pos [{'搭公車': 'True', '。': 'True', '討厭': 'True'}, 'neg']
correct: neg [{'期待': 'True', '研討會': 'True', '。': 'True', '參加': 'True', '不': 'True'}, 'neg']
accuracy is 0.5
predict ['neg', 'pos', 'pos', 'neg']


# 演算法3:LogisticRegression

In [37]:
print ('LogisticRegression-------------------')
sco=score(LogisticRegression(),train,data)
print('accuracy is', sco[0])
print ('predict', sco[1])

LogisticRegression-------------------
incorrect: neg [{'喜歡': 'True'}, 'pos']
correct: pos [{'最要': 'True', '。': 'True', '好': 'True', '朋友': 'True'}, 'pos']
incorrect: pos [{'搭公車': 'True', '。': 'True', '討厭': 'True'}, 'neg']
correct: neg [{'期待': 'True', '研討會': 'True', '。': 'True', '參加': 'True', '不': 'True'}, 'neg']
accuracy is 0.5
predict ['neg', 'pos', 'pos', 'neg']


C:\Users\brat\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


# 演算法4:SVC

In [38]:
print ('SVC-------------------')
sco=score(SVC(),train,data)
print('accuracy is', sco[0])
print ('predict', sco[1])

SVC-------------------
incorrect: neg [{'喜歡': 'True'}, 'pos']
correct: pos [{'最要': 'True', '。': 'True', '好': 'True', '朋友': 'True'}, 'pos']
incorrect: pos [{'搭公車': 'True', '。': 'True', '討厭': 'True'}, 'neg']
correct: neg [{'期待': 'True', '研討會': 'True', '。': 'True', '參加': 'True', '不': 'True'}, 'neg']
accuracy is 0.5
predict ['neg', 'pos', 'pos', 'neg']


C:\Users\brat\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


# 演算法5:LinearSVC

In [39]:
print ('LinearSVC-------------------')
sco=score(LinearSVC(),train,data)
print('accuracy is', sco[0])
print ('predict', sco[1])

LinearSVC-------------------
incorrect: neg [{'喜歡': 'True'}, 'pos']
correct: pos [{'最要': 'True', '。': 'True', '好': 'True', '朋友': 'True'}, 'pos']
incorrect: pos [{'搭公車': 'True', '。': 'True', '討厭': 'True'}, 'neg']
correct: neg [{'期待': 'True', '研討會': 'True', '。': 'True', '參加': 'True', '不': 'True'}, 'neg']
accuracy is 0.5
predict ['neg', 'pos', 'pos', 'neg']


# 演算法6:NuSVC

In [40]:
print ('NuSVC-------------------')
sco=score(NuSVC(),train,data)
print('accuracy is', sco[0])
print ('predict', sco[1])

NuSVC-------------------
incorrect: neg [{'喜歡': 'True'}, 'pos']
correct: pos [{'最要': 'True', '。': 'True', '好': 'True', '朋友': 'True'}, 'pos']
incorrect: pos [{'搭公車': 'True', '。': 'True', '討厭': 'True'}, 'neg']
correct: neg [{'期待': 'True', '研討會': 'True', '。': 'True', '參加': 'True', '不': 'True'}, 'neg']
accuracy is 0.5
predict ['neg', 'pos', 'pos', 'neg']


C:\Users\brat\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
